Philosophy Stack Exchange Questions + Selected Answers

Links of the ten selected questions: 


1.   https://philosophy.stackexchange.com/questions/38375/logical-fallacy-x-is-bad-y-is-worse-thus-x-is-not-bad
2.  https://philosophy.stackexchange.com/questions/77/can-you-prove-anything-in-philosophy




In [63]:
from post_parser_record import PostParserRecord
import re
import string

In [64]:
def collect_questions(post_reader, question_list):
  question_info = {}
  for question_id in question_list:
    question = post_reader.map_questions[question_id]
    title = question.title
    # cleans the question body
    body = question.body
    cleanr = re.compile('<.*?>|\n')
    body = re.sub(cleanr, ' ', body)
    body = body.replace('&amp;', '&')
    topTag = question.tags[0]
    accepted_answer_id = question.accepted_answer_id
    answer = post_reader.map_just_answers[accepted_answer_id]
    ans_body = answer.body
    ans_body = re.sub(cleanr, '', ans_body)
    ans_body = ans_body.replace('&amp;', '&')
    question_info[question_id] = [title, body, title + " " + body, topTag, ans_body]
  return question_info


In [65]:
post_file_path = "/content/Posts.xml"
post_reader = PostParserRecord(post_file_path)
question_ids = [38375, 77]
question_info = collect_questions(post_reader, question_ids)

In [66]:
#Showing accepted answers
for question, info in question_info.items():
  print(info[4])

This is (at least a subset of) the fallacy of relative privation.From Wikipedia >> List of Fallacies >> Red herring fallacies  Fallacy of relative privation ("not as bad as") – dismissing an  argument or complaint due to the existence of more important problems  in the world, regardless of whether those problems bear relevance to  the initial argument. For example, First World problem.RationalWiki adds  [Use of this fallacy is] popular with people who know perfectly well  they're doing something wrong; being fully aware that they're doing  something wrong, they feel compelled to attempt to justify it and do  so by pointing to other (usually worse) actions.A hypernym (again, RationalWiki) (emphasis mine) appears to be  Moral equivalence is a form of equivocation and a fallacy of relevance often used in political debates. It seeks to draw  comparisons between different, often unrelated things, to make a point  that one is just as bad as the other or just as good as the other. It  may be 

Question Title, Question Body, Question Top Tag, Question Selected Answer